# Update Refit and Evaluate Test

- load airlines
- define forecaster
- wrap in update*
- use evaluate
- visualise the forecasts as well to check

from sktime.datasets load

In [ ]:
from sktime.datasets import load_airline
from sktime.utils.plotting import plot_series
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.stream import (
    UpdateEvery,
    UpdateRefitsEvery,
    DontUpdate
)
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    SlidingWindowSplitter
)
from sktime.forecasting.model_evaluation import evaluate

import numpy as np
import pandas as pd


In [ ]:
y = load_airline()

# plotting for visualization
plot_series(y)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

need to change y to period index for this to run

In [ ]:
y = y.reset_index(drop=True)
y

doesn't seem to work anyway

naive forecaster is runs with updateevery, but updates every time whilst in evaulate with strategy='update'. Warning tells us that this is expected.

In [ ]:
mod = NaiveForecaster()
# up_mod = UpdateEvery(mod, update_interval=pd.Timedelta(31, "D"))
up_mod = UpdateEvery(mod, update_interval=3)
fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)
# cv = SlidingWindowSplitter(fh=fh, window_length=12, step_length=1)

df = evaluate(
    forecaster=up_mod,
    y=y,
    cv=cv,
    strategy="update",
    return_data=True,
    error_score='raise'
)


fig, ax = plot_series(
    y.iloc[1:20],
    df["y_pred"].iloc[0],
    df["y_pred"].iloc[1],
    df["y_pred"].iloc[2],
    df["y_pred"].iloc[3],
    df["y_pred"].iloc[4],
    df["y_pred"].iloc[5],
    markers=["o", "", "", "", "", "", ""],
    labels=["y_true"] + ["y_pred (Backtest " + str(x) + ")" for x in range(6)],
)
ax.legend()


Trying to use naive forecaster with UpdateRefitsEvery

In [ ]:
mod = NaiveForecaster()
# up_mod = UpdateEvery(mod, update_interval=pd.Timedelta(31, "D"))
up_mod = UpdateRefitsEvery(mod, refit_interval=3)
fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)
# cv = SlidingWindowSplitter(fh=fh, window_length=12, step_length=1)

df = evaluate(
    forecaster=up_mod,
    y=y,
    cv=cv,
    #strategy="update",
    return_data=True,
    error_score='raise'
)


fig, ax = plot_series(
    y.iloc[1:20],
    df["y_pred"].iloc[0],
    df["y_pred"].iloc[1],
    df["y_pred"].iloc[2],
    df["y_pred"].iloc[3],
    df["y_pred"].iloc[4],
    df["y_pred"].iloc[5],
    markers=["o", "", "", "", "", "", ""],
    labels=["y_true"] + ["y_pred (Backtest " + str(x) + ")" for x in range(6)],
)
ax.legend()

In [ ]:
from sktime.forecasting.arima import ARIMA

In [ ]:
mod = ARIMA(order=(1,0,0))
# up_mod = UpdateEvery(mod, update_interval=pd.Timedelta(31, "D"))
up_mod = UpdateEvery(mod, update_interval=3)
fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)
# cv = SlidingWindowSplitter(fh=fh, window_length=12, step_length=1)

df = evaluate(
    forecaster=up_mod,
    y=y,
    cv=cv,
    strategy="update",
    return_data=True,
    error_score='raise'
)


fig, ax = plot_series(
    y.iloc[1:20],
    df["y_pred"].iloc[0],
    df["y_pred"].iloc[1],
    df["y_pred"].iloc[2],
    df["y_pred"].iloc[3],
    df["y_pred"].iloc[4],
    df["y_pred"].iloc[5],
    markers=["o", "", "", "", "", "", ""],
    labels=["y_true"] + ["y_pred (Backtest " + str(x) + ")" for x in range(6)],
)
ax.legend()


In [ ]:
mod = ARIMA(order=(1,0,0))
# up_mod = UpdateEvery(mod, update_interval=pd.Timedelta(31, "D"))
up_mod = UpdateRefitsEvery(mod)
fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)
# cv = SlidingWindowSplitter(fh=fh, window_length=12, step_length=1)

df = evaluate(
    forecaster=up_mod,
    y=y,
    cv=cv,
    #strategy="update",
    return_data=True,
    error_score='raise'
)


fig, ax = plot_series(
    y.iloc[1:20],
    df["y_pred"].iloc[0],
    df["y_pred"].iloc[1],
    df["y_pred"].iloc[2],
    df["y_pred"].iloc[3],
    df["y_pred"].iloc[4],
    df["y_pred"].iloc[5],
    markers=["o", "", "", "", "", "", ""],
    labels=["y_true"] + ["y_pred (Backtest " + str(x) + ")" for x in range(6)],
)
ax.legend()


In [ ]:
mod = ARIMA(order=(1,0,0))
# up_mod = UpdateEvery(mod, update_interval=pd.Timedelta(31, "D"))
up_mod = UpdateRefitsEvery(mod, refit_interval=3)
fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)
# cv = SlidingWindowSplitter(fh=fh, window_length=12, step_length=1)

cv = list(cv.split(y))
for i in range(len(cv)):
    
    if not up_mod.is_fitted:
        indx_train = cv[i][0]
        y_train = y.iloc[indx_train].copy()
        up_mod.fit(y_train)
    else:
        indx_train = np.setxor1d(cv[i][0], cv[i-1][0])
        # todo: update params every X iters
        y_new = y.iloc[indx_train].copy()
        up_mod.update(
            y_new,
        )

    print(up_mod.forecaster_.get_fitted_params())

In [ ]:
cv[2]

Ok that doesn't seem to be working

    - let's try in a loop type evaluation
    - the problem is the datetime index!

In [ ]:
ok

In [ ]:
mod = NaiveForecaster()

# forecaster = UpdateRefitsEvery(mod, refit_interval=3)

# forecaster = UpdateEvery(mod, update_interval=3)
# forecaster = UpdateEvery(mod, update_interval=pd.Timedelta(31, "D"))
# forecaster = UpdateEvery(mod, update_interval=pd.DateOffset(months=3))

forecaster = UpdateEvery(mod, update_interval=None)

fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)
splits = list(cv.split(y))

y_pred = list()

for i in range(len(splits)):

    if not forecaster.is_fitted:
        indx_train = splits[i][0]
        forecaster.fit(y.iloc[indx_train])
    else:
        indx_train = np.setxor1d(splits[i][0], splits[i-1][0])
        # todo: update params every X iters
        forecaster.update(
            y.iloc[indx_train],
            update_params=True
        )
    # get target and issue times
    # fh_loop = splits[i][1]
    # fh_loop = y.iloc[fh_loop].index
    # print(fh_loop)

    # issue forecast
    tmp = forecaster.predict(fh=fh).rename('y_pred').to_frame()
    tmp.index.name = 'target_time'
    tmp['issue_time'] =  forecaster.cutoff

    y_pred.append(tmp)

y_pred = pd.concat(y_pred)
y_pred = y_pred.reset_index().set_index(['issue_time', 'target_time'])
y_pred




In [ ]:
mod = NaiveForecaster()

forecaster = UpdateEvery(mod, update_interval=3)

fh = np.arange(1, 7)
cv = ExpandingWindowSplitter(fh=fh, initial_window=12, step_length=1)


forecaster.fit(y=y.iloc[0:12])

preds = forecaster.update_predict(
    y=y,
    cv=cv, 
).melt(ignore_index=False).dropna()

preds.columns = ['issue_time', 'y_pred']
preds.index.name = 'target_time'

preds = preds.reset_index().set_index(['issue_time', 'target_time'])
preds

Still not working by the looks of it....

In [ ]:
y_pred.loc[y_pred.index.get_level_values('issue_time').isin([11, 12, 13, 14, 15, 16])]